In [ ]:
import pandas as pd
import numpy as np
from clifpy import ClifOrchestrator
from clifpy.utils import apply_outlier_handling
import sys
import os

sys.path.append('../src')
from icu_stitching import stitch_icu_stays
from helper import find_intubation_times

# Setup output path

In [ ]:
# output
output_folder = "../output"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(f'{output_folder}/final', exist_ok=True)
os.makedirs(f'{output_folder}/final/graphs', exist_ok=True)

# Intermediate tables
intermediate_path = "../private_tables"
os.makedirs(intermediate_path, exist_ok=True)

# Initialization

In [ ]:
co = ClifOrchestrator(
        config_path="../config/config.yaml",
        stitch_encounter=True,
        stitch_time_interval=6
    )

# Load Data

## Required variables

- hospitalization: age_at_admission
- labs: pco2_arterial, ph_arterial, po2_arterial, wbc, lymphocytes_absolute, neutrophils_absolute, hemoglobin, platelet_count,
    albumin, bicarbonate, creatinine, bilirubin_total, ferritin, lactate, ldh, procalcitonin, crp
- vitals: temp_c, heart_rate, map, respiratory_rate, spo2
- respiratory_support: peep_set, fio2_set, plateau_pressure_obs
- medication_admin_continous: norepinephrine
- patient_assessments: gcs_eye, gcs_motor, RASS
- crrt_therapy: crrt_flag
- ecmo_mcs: ecmo_flag

In [ ]:
# defined required columns and filters for each table
table_requirements = {
    'hospitalization': {
        'columns': ['hospitalization_id', 'patient_id', 'admission_dttm', 'discharge_dttm', 'age_at_admission', 
                      'admission_type_category', 'discharge_category'],
        'filters': {}
    },
    'patient': {
        'columns': ['patient_id', 'race_category', 'ethnicity_category', 'sex_category', 'death_dttm'],
        'filters': {}
    },
    'adt': {
        'columns': ['hospitalization_id', 'hospital_id', 'in_dttm', 'out_dttm', 'location_category', 'location_type'],
        'filters': {}
    },
    'labs': {
        'columns': ['hospitalization_id', 'lab_result_dttm', 'lab_value_numeric', 'lab_category'],
        'filters': {'lab_category': ['co2_arterial', 'ph_arterial', 'po2_arterial', 'wbc', 
                                          'lymphocytes_absolute', 'neutrophils_absolute', 'hemoglobin', 
                                          'platelet_count', 'albumin', 'bicarbonate', 'creatinine', 'bilirubin_total', 
                                          'ferritin', 'lactate', 'ldh', 'procalcitonin', 'crp']
                                          }
    },
    'vitals': {
        'columns': ['hospitalization_id', 'recorded_dttm', 'vital_value', 'vital_category'],
        'filters': {'vital_category': ['temp_c', 'weight_kg', 'heart_rate', 'sbp', 'spo2', 'respiratory_rate']}
    },
    'respiratory_support': {
        'columns': ['hospitalization_id', 'recorded_dttm', 'device_category', 'mode_category', 'tracheostomy', 'peep_set', 'fio2_set', 'plateau_pressure_obs'],
        'filters': {}
    },
    'medication_admin_continuous': {
        'columns': ['hospitalization_id', 'admin_dttm', 'med_category', 'med_route_category', 'med_dose', 'med_dose_unit'],
        'filters': {'med_category': ['norepinephrine']}
    },
    'patient_assessments': {
        'columns': ['hospitalization_id', 'recorded_dttm', 'assessment_category', 'numerical_value'],
        'filters': {'assessment_category': ['gcs_eye', 'gcs_motor', 'RASS']}
    },
    'crrt_therapy': {
        'columns': ['hospitalization_id', 'recorded_dttm'],
        'filters': {}
    },
    'ecmo_mcs': {
        'columns': ['hospitalization_id', 'recorded_dttm'],
        'filters': {}
    }
}

In [ ]:
co.initialize(
    tables=table_requirements.keys(),
    columns={t: c['columns'] for t, c in table_requirements.items()},
    filters={t: c['filters'] for t, c in table_requirements.items()}
)

# 0. Data Pre-processing

### 0.1 Unit conversion

In [ ]:
preferred_units = {
    "norepinephrine": "mcg/kg/min"
}

co.convert_dose_units_for_continuous_meds(preferred_units=preferred_units)


# QC Check: Show unsuccessful conversion statistics
df = co.medication_admin_continuous.df_converted
total_rows = len(df)
status_counts = df['_convert_status'].value_counts()
unsuccessful_count = total_rows - status_counts.get('success', 0)
unsuccessful_pct = (unsuccessful_count / total_rows * 100) if total_rows > 0 else 0

print("=" * 60)
print(f"Total records: {total_rows:,}")
print(f"Unsuccessful conversions: {unsuccessful_count:,} ({unsuccessful_pct:.2f}%)")
print(f"Successful conversions: {status_counts.get('success', 0):,} ({100 - unsuccessful_pct:.2f}%)")
print()

if unsuccessful_count > 0:
    print("Top 5 Failure Reasons:")
    print("-" * 60)
    # Filter out 'success' and get top 5 failures
    top_failures = status_counts[status_counts.index != 'success'].head(5)
    for i, (reason, count) in enumerate(top_failures.items(), 1):
        pct_of_failures = (count / unsuccessful_count * 100)
        pct_of_total = (count / total_rows * 100)
        print(f"{i}. {reason}")
        print(f"   Count: {count:,} ({pct_of_failures:.1f}% of failures, {pct_of_total:.2f}% of total)")
        print()
print("=" * 60)

# Filter to keep only successful conversions
co.medication_admin_continuous.df_converted = df[df['_convert_status'] == 'success']

# 0.2 Encounter Stitching

In [ ]:
# Access the encounter mapping
encounter_mappings = co.get_encounter_mapping()
print(f"Created {encounter_mappings['encounter_block'].nunique()} encounter blocks")

# 1. Cohort Identification Criteria:
1. Age >= 18
2. Paitnet admitted to hospital between 2018-01-01 and 2022-12-31
3. Have mechanical ventilation > 48 hours during ICU stay 

In [ ]:
# ============================================================================
# STEP 1: Filter hospitalizations directly (age >= 18 AND admission between 2018-2022)
# ============================================================================
combined_mask = (
    (co.hospitalization.df['age_at_admission'] >= 18) &
    (co.hospitalization.df['admission_dttm'] >= '2018-01-01') & 
    (co.hospitalization.df['admission_dttm'] < '2023-01-01')
)
filtered_hosp_ids = co.hospitalization.df[combined_mask]['hospitalization_id'].unique()

# ============================================================================
# STEP 2: Filter to ICU encounters
# ============================================================================
icu_mask = co.adt.df['location_category'].str.lower().str.contains('icu', na=False)
icu_hosp_ids = co.adt.df[icu_mask]['hospitalization_id'].unique()

# ============================================================================
# STEP 3: Combine filters (Adult AND Date Range AND ICU)
# ============================================================================
final_hosp_ids = set(filtered_hosp_ids) & set(icu_hosp_ids)
final_hosp_ids = sorted(list(final_hosp_ids))

print(f"{'='*80}")
print(f"✅ FINAL ICU COHORT: {len(final_hosp_ids):,} hospitalizations")
print(f"{'='*80}\n")

# Filtering breakdown
print("Filtering breakdown:")
print(f"  Started with:                    {len(co.hospitalization.df):,}")
print(f"  After adult + date filters:      {len(filtered_hosp_ids):,}")
print(f"  After ICU filter:                {len(final_hosp_ids):,}")

Workflow to get vent > 48 hr
1. Get cohort adt
2. Link ICU stays with gaps of less than 6 hours
3. Filter patients have > 48 hour of intubation

In [ ]:
# Step 1: Filter icu cohort adt
icu_adt = co.adt.df[co.adt.df['hospitalization_id'].isin(final_hosp_ids)].copy()

# Step 2: Stitch icu stays
icu_adt_stitched = stitch_icu_stays(icu_adt, gap_hours=6)

# Step 3: Prepare respiratory support data
icu_resp = co.respiratory_support.df[
    co.respiratory_support.df['hospitalization_id'].isin(final_hosp_ids)
].copy()
icu_resp = icu_resp.merge(encounter_mappings, how='inner', on='hospitalization_id')

# Step 3.1: Identify intubation timepoints (±1 hour time window)
intub_times = find_intubation_times(icu_resp, time_window=1)

# Step 3.2: Match intubations to ICU stays
intub_icu_merged = intub_times.merge(
    icu_adt_stitched[['encounter_block', 'in_dttm', 'out_dttm', 'icu_rank']],
    on='encounter_block',
    how='inner'
)

# Step 3.3: Keep only intubations that occurred during an ICU stay
intub_during_icu = intub_icu_merged[
    (intub_icu_merged['recorded_dttm'] >= intub_icu_merged['in_dttm']) &
    (intub_icu_merged['recorded_dttm'] <= intub_icu_merged['out_dttm'])
].copy()

# Step 3.4: Calculate ventilation duration per ICU stay
vent_duration = intub_during_icu.groupby(['encounter_block', 'icu_rank']).agg({
    'recorded_dttm': ['min', 'max'],
    'in_dttm': 'first',
    'out_dttm': 'first'
}).reset_index()

vent_duration.columns = ['encounter_block', 'icu_rank', 'first_intub', 'last_intub', 'in_dttm', 'out_dttm']

# Calculate ventilation duration in hours
vent_duration['vent_duration_hours'] = (
    vent_duration['last_intub'] - vent_duration['first_intub']
).dt.total_seconds() / 3600

# Step 3.5: Filter to ICU stays with at least 48 hours of ventilation
prolonged_vent_icu = vent_duration[vent_duration['vent_duration_hours'] > 48].copy()

# Step 3.6: Map encounter_block back to hospitalization_id
prolonged_vent_icu = prolonged_vent_icu.merge(
    encounter_mappings[['encounter_block', 'hospitalization_id']].drop_duplicates(),
    on='encounter_block',
    how='left'
)

# Step 3.7: Get unique hospitalization_ids with prolonged ventilation
prolonged_vent_hosp_ids = prolonged_vent_icu['hospitalization_id'].unique()

# Step 3.8: Combine with original cohort to create final cohort
final_prolonged_vent_cohort = [hosp_id for hosp_id in final_hosp_ids if hosp_id in prolonged_vent_hosp_ids]

print(f"\n✅ Original ICU cohort: {len(final_hosp_ids):,} hospitalizations")
print(f"✅ With > 48 hours continuous ventilation: {len(final_prolonged_vent_cohort):,} hospitalizations")
print(f"   Excluded (no prolonged ventilation): {len(final_hosp_ids) - len(final_prolonged_vent_cohort):,}")
print(f"\n   Total ICU stays with prolonged vent: {len(prolonged_vent_icu):,}")
print(f"   Unique encounter_blocks: {prolonged_vent_icu['encounter_block'].nunique():,}")
print(f"   Mean ventilation duration: {prolonged_vent_icu['vent_duration_hours'].mean():.1f} hours")
print(f"   Median ventilation duration: {prolonged_vent_icu['vent_duration_hours'].median():.1f} hours")

# Data Preparation Workflow:
1. Aggregate daily average measurements and pivot to create one row per ICU-day
2. VAP start dttm: 
3. ecmo_flag: any ecmo recorded dttm in a icu-day then mark as 1
4. crrt_flag: any crrt recorded dttm in a icu-day then mark as 1

In [ ]:
# Get cohort tables 
cohort_labs = co.labs.df[co.labs.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()
cohort_vitals = co.vitals.df[co.vitals.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()
cohort_resp = co.respiratory_support.df[co.respiratory_support.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()
cohort_med_continuous = co.medication_admin_continuous.df[co.medication_admin_continuous.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()
cohort_assessments = co.patient_assessments.df[co.patient_assessments.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()
cohort_crrt = co.crrt_therapy.df[co.crrt_therapy.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()
cohort_ecmo = co.ecmo_mcs.df[co.ecmo_mcs.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()
cohort_hospitalization = co.hospitalization.df[co.hospitalization.df['hospitalization_id'].isin(final_prolonged_vent_cohort)].copy()